In [3]:
import sys
from PyQt5.QtCore import *
from PyQt5.QtGui import *
from PyQt5.QtWidgets import *
from PyQt5 import QtWidgets


In [5]:
class photo_demo(QtWidgets.QWidget):
    #signal
    _signal = QtCore.pyqtSignal()
    def __init__(self,parent = None):
        super(photo_demo,self).__init__(parent)
        vertical_layout = QVBoxLayout()
        grid_layout = QGridLayout()
        grid_layout.setSpacing(10)
        
        # set the size of window
        self.resize(300,300)
        
        self.button = QPushButton()
        self.button.clicked.connect(self.photo_load)
        self.button.setText('Loading Photo from File')
        grid_layout.addWidget(self.button,1,0)
        
        self.label = QLabel()
        self.label.setText('Image')
        grid_layout.addWidget(self.label,1,1)
        
        self.label_2 = QLabel()
        self.label_2.setText('Output:')
        grid_layout.addWidget(self.label_2,4,0)
        
        self.label_3 = QLabel()
        self.label_3.setText('1')
        grid_layout.addWidget(self.label_3,4,1)
        
#         self.content = QTextEdit()
#         layout.addWidget(self.content)
        
        self.setWindowTitle('Guess Photo!')
        self.setLayout(grid_layout)
        
        #signal
        self._signal.connect(self.mysignal)
    
    def photo_load(self):
        print('Loading..')
        result = QMessageBox.warning(self,('Warning'),('''Please be certain kinds of photos'''),QMessageBox.StandardButtons(QMessageBox.Yes),QMessageBox.Yes)
        fname, _ = QFileDialog.getOpenFileName(self,'Choose Photos',r'C:/Users/xyuhubert/Desktop/python work/Github','Image files(*.jpg *.gif *.png)')
        print(fname)
        self.label.setPixmap(QPixmap(fname))
        self._signal.emit()
#         print(type(fname))
#         pixmap = QPixmap(C:\Users\xyuhubert\Desktop\python work\Github\tiger.jpg)
#         self.label.setPixmap(pixmap)
    def mysignal(self, parameter):
        print(parameter)
    
if __name__ == '__main__':
# must construct a QApplication before QWidget
    app = QApplication(sys.argv) 
    photoload = photo_demo()
    photoload.show()
    sys.exit(app.exec_())
        

Loading..
C:/Users/xyuhubert/Desktop/python work/Github/data/testing/img/img-0.png


SystemExit: 0

C:\Python\lib\site-packages\IPython\core\interactiveshell.py:2971: UserWarning: To exit: use 'exit', 'quit', or Ctrl-D.
  warn("To exit: use 'exit', 'quit', or Ctrl-D.", stacklevel=1)
